In [1]:
from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
import multiprocessing

from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz,export_text, plot_tree
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB


from sklearn.datasets import load_boston, load_breast_cancer,load_iris
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC #SVM
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.metrics import f1_score


현대 자동차 데이터를 이용  
가격:1135   
년식:2015  
연비: 15  
마력:100  
토크:13.6  
연료:가솔린  
하이브리드:0  
배기량:1368  
중량:1035  
변속기:수동  

인경우 종류( 준중형, 소형, 대형)를 예측하시요.  
KNN, decision tree, naive_bayes, svc 각각의 모델에 대해  
0. 각모델별 예측값 구하시요.
1. f1 score(train, test) , confusion matrix,
를 구하시요
2. 각모델별 train ,test f1 를  bar 그래프로 그리고
가장높은 f1 score 의 모델을 확인하시요.


In [2]:
car_df = pd.read_excel('data5/hyundaiCar.xlsx')
car_df.head()

,가격,년식,종류,연비,마력,토크,연료,하이브리드,배기량,중량,변속기
0,1885,2015,준중형,11.8,172,21.0,가솔린,0,1999,1300,자동
1,2190,2015,준중형,12.3,204,27.0,가솔린,0,1591,1300,자동
2,1135,2015,소형,15.0,100,13.6,가솔린,0,1368,1035,수동
3,1645,2014,소형,14.0,140,17.0,가솔린,0,1591,1090,자동
4,1960,2015,대형,9.6,175,46.0,디젤,0,2497,1990,자동


In [3]:
x_data = car_df.drop('종류', axis=1)
y_data = car_df[['종류']]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, stratify=y_data, random_state=1)
ct = make_column_transformer((OneHotEncoder(), ['연료','변속기']), remainder='passthrough')

In [4]:
# KNN
model_KNN = make_pipeline(ct, StandardScaler(), KNeighborsClassifier())
param_value = {'kneighborsclassifier__n_neighbors':[1,2,3,4,5]}
gs_KNN = GridSearchCV(model_KNN, param_grid=param_value)
gs_KNN.fit(x_train, y_train)

C:\Users\medici\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\medici\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\medici\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\medici\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         ['연료',
                                                                          '변속기'])])),
                                       ('standardscaler', StandardScaler()),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier())]),
             param_grid={'kneighborsclassifier__n_neighbors': [1, 2, 3, 4, 5]})

In [5]:
# 예측값
print('예측값: ',
      gs_KNN.best_estimator_.predict(pd.DataFrame([[1135,2015,15,100,13.6,'가솔린',0,1368,1035,'수동']],
                                                  columns=x_data.columns.values)))

y_pred_train = gs_KNN.best_estimator_.predict(x_train)
y_pred_test = gs_KNN.best_estimator_.predict(x_test)

# f1 score
print('train_f1: ', f1_score(y_train, y_pred_train,average='macro'))
print('test_f1: ', f1_score(y_test, y_test ,average='macro'))

예측값:  ['소형']
train_f1:  1.0
test_f1:  1.0


In [6]:
# Decision Tree
model_DT = make_pipeline(ct, StandardScaler(), DecisionTreeClassifier())
param_value = {'decisiontreeclassifier__criterion':['entropy','gini']}
gs_DT = GridSearchCV(model_DT, param_grid=param_value)
gs_DT.fit(x_train, y_train)

# 예측값
print('예측값: ', gs_DT.predict(pd.DataFrame([[1135,2015,15,100,13.6,'가솔린',0,1368,1035,'수동']], columns=x_data.columns.values)))

y_pred_train = gs_DT.best_estimator_.predict(x_train)
y_pred_test = gs_DT.best_estimator_.predict(x_test)

# f1 score
print('train_f1: ', f1_score(y_train, y_pred_train, average='macro'))
print('test_f1: ', f1_score(y_test, y_test, average='macro'))

예측값:  ['소형']
train_f1:  1.0
test_f1:  1.0


In [7]:
# Naive Bayes
model_NB = make_pipeline(ct, StandardScaler(), GaussianNB())
model_NB.fit(x_train, y_train)

# 예측값
print('예측값: ', model_NB.predict(pd.DataFrame([[1135,2015,15,100,13.6,'가솔린',0,1368,1035,'수동']], columns=x_data.columns.values)))

y_pred_train = model_NB.predict(x_train)
y_pred_test = model_NB.predict(x_test)

# f1 score
print('train_f1: ', f1_score(y_train, y_pred_train, average='macro'))
print('test_f1: ', f1_score(y_test, y_test, average='macro'))

예측값:  ['소형']
train_f1:  0.49016961299569994
test_f1:  1.0


C:\Users\medici\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [8]:
# SVC
model_SVC = make_pipeline(ct, StandardScaler(), SVC())
param_value = {'svc__kernel' : ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
              'svc__C': list(range(0,11)), 'svc__gamma':['scale','auto']}
gs_SVC = GridSearchCV(model_SVC, param_grid=param_value, scoring='f1_macro')
gs_SVC.fit(x_train, y_train)


C:\Users\medici\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\medici\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\medici\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\medici\anaconda3\lib\site-packages\sklearn\pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\medici\anaconda3\lib\site-packages\sklearn\svm\_base.py", line 226, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  Fi

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         ['연료',
                                                                          '변속기'])])),
                                       ('standardscaler', StandardScaler()),
                                       ('svc', SVC())]),
             param_grid={'svc__C': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'svc__gamma': ['scale', 'auto'],
                         'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid',
                                         'precomputed']},
             scoring='f1_macro')

In [9]:
# 예측값
print('예측값: ', gs_SVC.best_estimator_.predict(pd.DataFrame([[1135,2015,15,100,13.6,'가솔린',0,1368,1035,'수동']], columns=x_data.columns.values)))

y_pred_train = gs_SVC.best_estimator_.predict(x_train)
y_pred_test = gs_SVC.best_estimator_.predict(x_test)

# f1 score
print('train_f1: ', f1_score(y_train, y_pred_train, average='macro'))
print('test_f1: ', f1_score(y_test, y_test, average='macro'))

예측값:  ['소형']
train_f1:  0.9647727272727273
test_f1:  1.0
